In [1]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq transformers --progress-bar off
!pip install -qqq datasets --progress-bar off
!pip install -qqq peft --progress-bar off
!pip install -qqq bitsandbytes --progress-bar off
!pip install -qqq accelerate --progress-bar off
!pip install -qqq trl --progress-bar off
!pip install -qqq ipython --progress-bar off

In [2]:
import json
import re
from pprint import pprint

import pandas as pd
import torch
from datasets import Dataset, load_dataset
from huggingface_hub import notebook_login
from peft import LoraConfig, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from trl import SFTTrainer

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
MODEL_NAME = "meta-llama/Llama-2-7b-hf"

In [3]:
notebook_login()

In [4]:
dataset = load_dataset("AsphyXIA/baarat-hin-v1")
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/5062893 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['src', 'tgt'],
        num_rows: 5062893
    })
})

In [5]:
dataset = dataset["train"]

In [6]:
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME,load_in_4bit=True,torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME,torch_dype=torch.float16)

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [8]:
torch.cuda.empty_cache()

In [7]:
from transformers import PreTrainedTokenizerFast

tokenizer_hin = PreTrainedTokenizerFast(tokenizer_file="hin-sml-wdpc.json")

In [9]:
from transformers import Trainer, TrainingArguments
from transformers import AutoModelForCausalLM
from datasets import load_dataset

mapping = {}
for token, id in tokenizer.get_vocab().items():
   if token in tokenizer_hin.get_vocab():
       mapping[id] = tokenizer_hin.get_vocab()[token]

def adjust_ids(ids):
   return [mapping[id] if id in mapping else id for id in ids]



In [10]:
original_vocab = tokenizer.get_vocab()
new_vocab = tokenizer_hin.get_vocab()

# Merge the vocabularies
merged_vocab = {**original_vocab, **new_vocab}


In [11]:
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors
from tqdm import tqdm

# Create a new BPE model
bpe_model = models.BPE()

# Create a new tokenizer with the merged vocabulary
merged_tokenizer = Tokenizer(bpe_model)

# Add all tokens from both tokenizers to the new tokenizer
for token, id in tqdm(tokenizer.get_vocab().items()):
   merged_tokenizer.add_tokens([token])

for token, id in tqdm(tokenizer_hin.get_vocab().items()):
   merged_tokenizer.add_tokens([token])


100%|██████████| 20000/20000 [53:33<00:00,  6.22it/s]


In [16]:
import json
import os
# with open(os.path.join(directory, 'vocab.json'), 'w') as fp:
#    json.dump(merged_tokenizer.get_vocab(), fp)

# Save the tokenizer configuration
merged_tokenizer.save("./my_merged_tokenizer/tokenizer.json")


In [17]:
!zip -r /content/merged_tokenizer.zip /content/my_merged_tokenizer

  adding: content/my_merged_tokenizer/ (stored 0%)
  adding: content/my_merged_tokenizer/vocab.json (deflated 67%)
  adding: content/my_merged_tokenizer/tokenizer.json (deflated 95%)
  adding: content/my_merged_tokenizer/.ipynb_checkpoints/ (stored 0%)


In [19]:
from tokenizers import Tokenizer

directory = "./my_merged_tokenizer"
merged_tokenizer = Tokenizer.from_file(os.path.join(directory, 'tokenizer.json'))


In [22]:
encoded = merged_tokenizer.encode("Hello, world! एक मछली सारे तालाब को गंदा कर देती है |")
print(encoded.ids)

[25834, 24514, 30701, 1245, 37597, 50639, 42671, 50720, 40446, 50173, 44011, 43473, 36418, 48020, 13046]


In [23]:
decoded = merged_tokenizer.decode(encoded.ids)
print(decoded)

Hello , world ! एक मछली सारे तालाब को गं दा कर देती है |
